In [8]:
#cifar 10 dataset
import torch
import torchvision
import torchvision.transforms as transforms

from tensorflow.keras import datasets, layers, models
transform = transforms.Compose(
    [transforms.ToTensor()])

batch_size = 4

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
# trainset = torchvision.datasets.CIFAR10(root='datasets/CIFAR10/', train=True,
#                                         download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
#                                           shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(root='datasets/CIFAR10/', train=False,
#                                        download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
#                                          shuffle=False, num_workers=2)

# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


/home/nle/anaconda3/envs/abc/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nle/anaconda3/envs/abc/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nle/anaconda3/envs/abc/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nle/anaconda3/envs/abc/lib/python3.7/site-packages/tensorf

170500096/170498071 [==============================] - 21s 0us/step


In [2]:
"""
Load a Marabou net for CIFAR 10
"""
from maraboupy import Marabou, MarabouCore, MarabouUtils
from onnx2pytorch import ConvertModel
import onnx
import numpy as np

PATH='datasets/CIFAR10/marabou-cifar10/nets/cifar10_large.onnx'
marabou_network = Marabou.read_onnx(PATH)
onnx_model = onnx.load(PATH)
pytorch_model = ConvertModel(onnx_model)
print(pytorch_model)




/home/nle/opt/Marabou/maraboupy/Marabou.py:27: UserWarning: Tensorflow parser is unavailable because tensorflow package is not installed
  warnings.warn("Tensorflow parser is unavailable because tensorflow package is not installed")


ConvertModel(
  (Conv_sequential_2/conv2d_4/BiasAdd:0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2))
  (Relu_sequential_2/conv2d_4/Relu:0): ReLU(inplace=True)
  (Conv_sequential_2/conv2d_5/BiasAdd:0): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
  (Relu_sequential_2/conv2d_5/Relu:0): ReLU(inplace=True)
  (Reshape_sequential_2/flatten_2/Reshape:0): Reshape(shape=[  -1 2304])
  (MatMul_sequential_2/dense_6/BiasAdd:0): Linear(in_features=2304, out_features=128, bias=True)
  (Relu_sequential_2/dense_6/Relu:0): ReLU(inplace=True)
  (MatMul_sequential_2/dense_7/BiasAdd:0): Linear(in_features=128, out_features=64, bias=True)
  (Relu_sequential_2/dense_7/Relu:0): ReLU(inplace=True)
  (MatMul_Identity:0): Linear(in_features=64, out_features=10, bias=True)
)


/home/nle/anaconda3/envs/abc/lib/python3.7/site-packages/onnx2pytorch/convert/layer.py:25: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1627336316785/work/torch/csrc/utils/tensor_numpy.cpp:143.)
  layer.weight.data = torch.from_numpy(numpy_helper.to_array(weight))


In [16]:
"""
Check accuracy of the loaded model. Expected: 74.16%
"""
from sklearn.metrics import confusion_matrix, accuracy_score
true_labels = []
pred_labels = []
for idx in range(len(test_images)):
    input = (test_images[idx]/255).astype(np.float32).reshape(1,32,32,3)
    # print(input, input.shape)
    pred = pytorch_model(torch.Tensor(input))
    # print(pred, np.argmax(pred), labels[idx])
    true_labels.append(test_labels[idx])
    if pred is not None:
        pred_labels.append(torch.argmax(pred))
    else:
        pred_labels.append(-1)
print(confusion_matrix(true_labels, pred_labels))
print(accuracy_score(true_labels, pred_labels))

[[775  16  33  21  18   4   6  10  67  50]
 [ 12 833   3  10   3   2  11   2  24 100]
 [ 57   4 560  85 122  75  51  26   6  14]
 [ 10   5  47 563  66 212  42  22  14  19]
 [ 19   3  34  67 734  33  29  64  12   5]
 [ 14   2  29 154  55 687  12  37   5   5]
 [  7   5  24  77  50  28 796   3   4   6]
 [ 10   1  19  45  60  74   5 773   2  11]
 [ 34  31   7  18   4   3   3   5 863  32]
 [ 24  64   6  20   7   6   3  12  26 832]]
0.7416


In [ ]:
idx = 0
input = (test_images[idx]/255).astype(np.float32).reshape(1,32,32,3)
print(marabou_network.evaluate(input))
print(pytorch_model(torch.Tensor(input)))
marabou_network.reluList

In [20]:
print(len(test_images))

10000
